# Architecture simulator-based power modeling


In this notebook, we will demonstrate the process of launching the architecture simulator to simulate the application on the provided architecture and retrieve event counter information. Additionally, we will explain how to utilize the simulation outputs to estimate power consumption.


## Setting up and building gem5-X
The gem5-X is already configured and built in the VM. If you want to build it from scratch, you can follow the instructions in the [gem5-X repository](https://github.com/gem5-X/gem5-X)


## Prepare the gem5-X environment


The gem5-X environment is created and configured with the provided docker image and container, and the container is already built in the VM, called **gem5x-container**. You can directly go to next section to launch the gem5-X environment.
But if you want to create the docker container from scratch, you can use the following command to create the container and share the local files between the docker container and the host.

```
docker run --name gem5x-container -it -v /home/student/Documents/gem5-X:/app gem5x
```

**docker run**: This is the command used to run a Docker container.

**--name gem5x-container**: This option sets the name of the Docker container as "gem5x-container". Containers can be identified and managed using their names.

**-it**: These option keeps the container attached to the terminal. This allows you to interact with the container's shell and see the output directly.

**-v /home/student/Documents/gem5-X:/app**: This option creates a volume mount between the host machine and the container. It maps the directory /home/student/Documents/gem5-X on the host machine to the /app directory inside the container. This means that any files or directories in /home/student/Documents/gem5-X can be accessed from within the container at /app.

**gem5x**: This is the name of the Docker image that will be used to create the container.

By running this command, a Docker container named "gem5x-container" will be created using the "gem5x" image. The container will be attached to the terminal, and the /home/student/Documents/gem5-X directory on the host machine will be accessible within the container at /app.


## EX1: Launch your first gem5-X simulation

Run the follow commnad to start the gem5x container
```
docker start gem5x-container
```

and attach to the container with the follow command
```
docker exec -it gem5x-container bash
```

Now you are inside the container. We have prepared several scripts to launch the gem5-X simulation. For instance, **“/app/dtm_course_materials/day3/ex1_runGem5-X.sh** can start the gem5-X simulation.

```
#!/bin/bash

gem5_path=/app/gem5-X

${gem5_path}/build/ARM/gem5.fast \
-d ./output_ex1 \
${gem5_path}/configs/example/fs.py \
--cpu-clock=1GHz \
--kernel=vmlinux \
--machine-type=VExpress_GEM5_V1 \
--dtb-file=${gem5_path}/system/arm/dt/armv8_gem5_v1_1cpu.dtb \
-n 1 \
--disk-image=gem5_ubuntu16.img \
--caches \
--l2cache \
--l1i_size=32kB \
--l1d_size=32kB \
--l2_size=1MB \
--l2_assoc=2 \
--mem-type=DDR4_2400_4x16 \
--mem-ranks=4 \
--mem-size=1GB \
--sys-clock=1600MHz
```

 and you can run the gem5-X simulation with the follow command
```
cd app/scripts/
bash ex1_runGem5-X.sh
```

Because we are launching a full system Linux simulation with gem5-X, so you can connect to its terminal. Now open another terminal tab and run the follow command to connect to the gem5-X terminal. Note that you still need to be in the gem5x-container to connect to the gem5 terminal, so you need to first attach to the container and then run the m5term.
``` 
docker exec -it gem5x-container bash
./app/gem5-X/util/term/m5term 127.0.0.1 3456
```

You can see the ubuntu is booting. However, this can take 10 to 20 minutes to complete. We will skip this process by using the checkpointing feature of gem5-X.




## EX2: Using gem5-X’s checkpointing feature

A checkpoint in gem5-X captures the entire state of the simulated system at a specific point in time, including the register values, memory contents, cache state, and microarchitectural details. It allows users to save the state of a running simulation and later restore it to continue execution from the exact same point. This is particularly useful for debugging, performance analysis, and architecture exploration purposes.

We provide a script to run the gem5-X simulation with checkpointing. The script is **ex2_runGem5-X-checkpoint.sh** with one additional argument **-r 1**.

You can run the script with the following command.
```
<!-- Stop the previous simulation first, then -->
bash ex2_runGem5-X-checkpoint.sh
<!-- connect to the m5term  -->
```

Now you can see that ubuntu is already booted. It is a full-function ubuntu thanks to the full system simulation with gem5-X. For example, you can run "ls" and the "hello world" program in the terminal.

To save your own checkpoint, you can use the following command inside the gem5-X terminal:
```
m5 checkpoint
```



## EX2.1: Profiling the target application

We have already put a simple application (sum_bench) in the gem5-X environment. It is a simple benchmark to test how fast the CPU can add up numbers. You can run the following command to run and profile the application.
```
m5 resetstats; ./sum_bench 2000000; m5 dumpstats
```

**m5 resetstats**: This command resets the statistics counters within the simulator, allowing you to start measuring performance or other metrics from a clean slate. By resetting the stats, any previous measurements are discarded.

**./sum_bench 2000000**: This command executes "sum_bench" with an argument of "2000000". You can put everthing you want to profile here.

**m5 dumpstats**: This command dumps the current statistics into stats.txt.

It may take 1 or 2 minutes, based on your machine. Now, let's check the output results in stat.txt


## EX2.2: Power modeling

Once we have the stats.txt file ready, we can use the power modeling technique introduced in the lecture to estimate the power consumption of the target application. We have prepared several code snipets for this.

In [ ]:
# Useful functions to calculate the energy consumption of the system
import pandas as pd
# Here is the code to parse the stats file and save it to a dictionary
def parse_stat_file(filename):
    data = {}
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('-') or not line:
                continue
            key, value = line.split('#')[0].split()[0:2]
            try:
                value = int(value)
            except ValueError:
                value = float(value)
            data[key] = value
    return data

# This function will use the parsed data and the power model to calculate the energy consumption of the system
# The basic idea is to check the event counts and multiply them with the enegy consumption of the event
# In this case, we are focusing on the core and the LLC
def energy_consumption(power_model, parsed_data, freq_run, llc_size):
    # get the index of the frequency in the power model
    power_loc=power_model.loc[power_model['Freq'] == freq_run].index[0]

    # get the number of cycles and idle cycles from the parsed data
    num_cycles=parsed_data['system.cpu.numCycles']
    idle_cycles=parsed_data['system.cpu.num_idle_cycles']

    # calculate the energy consumption of the core for active, idle and static power
    energy_active=(num_cycles-idle_cycles)*power_model['CoreActive'][power_loc]*10e-12

    energy_idle=idle_cycles*power_model['CoreIdle'][power_loc]*10e-12

    energy_static=num_cycles*power_model['CoreStatic'][power_loc]*10e-12

    # calculate llc read, write and leakage energy
    llc_read = parsed_data['system.l2.ReadReq_hits::total']+\
        parsed_data['system.l2.ReadExReq_hits::total']+\
        parsed_data['system.l2.ReadCleanReq_hits::total']+\
        parsed_data['system.l2.ReadSharedReq_hits::total']
    
    energy_llc_read=llc_read*power_model['LLCRead'][power_loc]*10e-12

    llc_write = parsed_data['system.l2.WritebackDirty_hits::total']+\
        parsed_data['system.l2.WritebackClean_hits::total']+\
        parsed_data['system.l2.WriteClean_hits::total']
    
    energy_llc_write=llc_write*power_model['LLCWrite'][power_loc]*10e-12

    energy_llc_leak=num_cycles*llc_size*power_model['LLCStatic'][power_loc]*10e-12

    # calculate the total energy
    total_energy=energy_active+energy_idle+energy_static+energy_llc_read+energy_llc_write+energy_llc_leak
    
    return total_energy

With the above predefined functions, you can run the following command to get the power consumption of the gem5-X running.

In [ ]:
# Load the power model csv file
power_model = pd.read_csv('power_model.csv', sep=',', header=0)
# Parse the stats file
filename = '/PATHTO/stats.txt'  # Replace with the actual filename
parsed_data = parse_stat_file(filename)

freq_run=1005
llc_size=1
energy = energy_consumption(power_model, parsed_data, freq_run, llc_size)
print("Runtime:", parsed_data['sim_seconds'])
print("Consumed enegy:", energy)

---
**NOTE**
The above analysis calculates the total consumed energy while running the target application. If you want to get the average power consumption during the execution, you can just divide the total energy by the execution time.
If you want to get the power trace, gem5-X supports dumping the runtime stats periodically. You can use the following commands to profile the application and get the stats trace:
```
m5 resetstats; #YOU_APPLICAITON; m5 dumpstats #DELAY #PERIOD
```

The m5 dumpstats command now save simulation statistics to a file in #DELAY nanoseconds delay and repeat this every #PERIOD nanoseconds.

In this case, you can get stats trace per #PERIOD nanoseconds. Combined with a power model, you can easily get the power trace.

---


## EX3: Explore the hardware configuration
Now, let's explore the power/energy consumption with different hardware parameters. For instance, Checkpoint is very useful when you only want to change the configurations of the hardware. For instance, let's change the frequency of the CPU with the script **ex3_runGem5-X-checkpoint.sh**.

```
<!-- Stop the previous simulation first, then -->
bash ex3_runGem5-X-checkpoint.sh
<!-- connect to the m5term  -->
```

Then you can refer to the command in EX2.1 to profile the application and estimate the power consumption. After that, please finish the following exercises on your own:
1. Select 10 different frequency points, and run the sum_bench application on each frequency point. Record the execution time and energy consumption.
2. Fix the CPU frequency at 1.005GHz, and change the cache size to investigate the possible execution time and energy impacts.



## Additional EX4: Modify the image with chroot

It's also possible to run your own benchmark in the gem5-X environment. You need to first put the target application and benchmark inside the Ubuntu image first. You can use the chroot command to do that.

```
<!-- Inside the VM, not the docker container -->
cd /home/student/Documents/gem5-X/gem5-X/full_system_images/disks
sudo mount -o loop,offset=$((2048*512)) gem5_ubuntu16.img local_mnt
sudo mount -o bind /proc local_mnt/proc
sudo mount -o bind /dev local_mnt/dev
sudo mount -o bind /dev/pts local_mnt/dev/pts
sudo mount -o bind /sys local_mnt/sys

cd local_mnt/
sudo chroot ./ 
```
Once it is finished, exit and umount the image
```
exit or ctrl+d

cd ..
sudo umount local_mnt/proc
sudo umount local_mnt/dev/pts
sudo umount local_mnt/dev
sudo umount local_mnt/sys
sudo umount local_mnt

```

Note that you need to boot the image with gem5-X from scratch (without using checkpoint) when you change the image.